# Halloween Challenge :) 
by Gregorio Nicora

In [152]:
import math
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy

In [153]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [154]:
NUM_POINTS = 100
NUM_SETS = NUM_POINTS
DENSITY = .3
ITERATIONS = 100

In [155]:
x = make_set_covering_problem(NUM_POINTS, NUM_SETS, DENSITY)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [156]:
SETS = x.toarray()

In [157]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

In [158]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [159]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print("SETS", SETS, current_state, fitness(current_state), sep="\n")
for step in range(ITERATIONS):
    new_state = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print("Step", step, ":", fitness(current_state))

SETS
[[ True  True False ... False  True False]
 [ True False False ...  True False  True]
 [False False False ... False  True  True]
 ...
 [False False  True ... False False  True]
 [False False False ... False False False]
 [False False False ...  True False  True]]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
(0, 0)
Step 0 : (25, -1)
Step 

# Simulated Annealing

In [160]:
def compute_p(fit_current_state, fit_new_state, temp):
    return math.exp(-abs(fit_current_state[0]-fit_new_state[0])/temp)

In [161]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print("SETS", SETS, current_state, fitness(current_state), sep="\n")
temp = ITERATIONS
for step in range(ITERATIONS):
    new_state = tweak(current_state)
    temp = temp - step * 0.005
    p = compute_p(fitness(current_state), fitness(new_state), temp)
    if ((fitness(new_state) >= fitness(current_state)) or random() < p) and temp > 0:
        current_state = new_state
        print("Step", step, ":", fitness(current_state))

SETS
[[ True  True False ... False  True False]
 [ True False False ...  True False  True]
 [False False False ... False  True  True]
 ...
 [False False  True ... False False  True]
 [False False False ... False False False]
 [False False False ...  True False  True]]
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
(0, 0)
Step 0 : (32, -1)
Step 

# Tabu Search